In [1]:
import pandas as pd
import numpy as np
game_log_path = './data/GL2021.txt'
retro_df_columns=['date', 'doubleheader_index','weekday', 'visitor', 'visitor_league', 'visitor_game_num', 'home', 'home_league', 'home_game_num', 'visitor_score', 'home_score', 'length_outs', 'day_night', 'completion']
df = pd.read_csv(game_log_path, header=0, names=retro_df_columns, usecols=range(len(retro_df_columns)))
# The never-made-up COL-ATL game is not here:
# df.loc[(df['date'] == 20210916)]

df['date'] = df['date'].astype(str)
df['completion_date'] = df['date']
df.loc[(~df['completion'].isnull()), 'completion_date'] = df['completion'].str.split(',').str[0]
df['completion_date'] = df['completion_date'].astype(str)

df.loc[(df['date'] != df['completion_date'])]

,date,doubleheader_index,weekday,visitor,visitor_league,visitor_game_num,home,home_league,home_game_num,visitor_score,home_score,length_outs,day_night,completion,completion_date
122,20210411,0,Sun,MIA,NL,9,NYN,NL,6,5,6,53,D,"20210831,NYC20,0,0,1",20210831
720,20210526,0,Wed,CIN,NL,47,WAS,NL,45,3,5,51,D,"20210527,,0,3,21",20210527
1366,20210717,0,Sat,MIA,NL,93,PHI,NL,91,2,4,58,D,"20210718,PHI13,2,2,54",20210718
1369,20210717,0,Sat,SDN,NL,95,WAS,NL,91,10,4,54,D,"20210718,WAS11,8,4,33",20210718
1421,20210721,2,Wed,SDN,NL,99,ATL,NL,95,6,5,42,N,"20210924,SAN02,5,4,27",20210924
1686,20210810,0,Tue,WAS,NL,113,NYN,NL,113,7,8,51,D,"20210811,NYC20,3,1,6",20210811


In [2]:
# adapted from sdvinay
def compute_standings(gms_played):
    margins = gms_played['visitor_score']-gms_played['home_score']
    winners = pd.Series(np.where(margins>0, gms_played['visitor'], gms_played['home']))
    losers  = pd.Series(np.where(margins<0, gms_played['visitor'], gms_played['home']))
    standings = pd.concat([winners.value_counts().rename('W'), losers.value_counts().rename('L')], axis=1)
    return standings.fillna(0)

compute_standings(df).sub(compute_standings(df.loc[(df['completion_date'] == '20211003')]), fill_value=0)

,W,L
ANA,76.0,85.0
ARI,51.0,110.0
ATL,87.0,73.0
BAL,52.0,109.0
BOS,91.0,70.0
CHA,93.0,68.0
CHN,70.0,90.0
CIN,82.0,79.0
CLE,79.0,82.0
COL,74.0,86.0


In [3]:
# adapted from sdvinay
#  Create a data frame with the league/division mappings, to use to determine playoff berths
divisions = pd.DataFrame({
'SFN': ['N','NW'],
'LAN': ['N','NW'],
'TBA': ['A','AE'],
'MIL': ['N','NC'],
'HOU': ['A','AW'],
'CHA': ['A','AC'],
'BOS': ['A','AE'],
'NYA': ['A','AE'],
'TOR': ['A','AE'],
'OAK': ['A','AW'],
'SEA': ['A','AW'],
'SDN': ['N','NW'],
'ATL': ['N','NE'],
'CIN': ['N','NC'],
'PHI': ['N','NE'],
'SLN': ['N','NC'],
'NYN': ['N','NE'],
'ANA': ['A','AW'],
'CLE': ['A','AC'],
'DET': ['A','AC'],
'CHN': ['N','NC'],
'COL': ['N','NW'],
'KCA': ['A','AC'],
'MIN': ['A','AC'],
'MIA': ['N','NE'],
'WAS': ['N','NE'],
'TEX': ['A','AW'],
'PIT': ['N','NC'],
'BAL': ['A','AE'],
'ARI': ['N','NW']
 }).T

divisions.columns = ['lg', 'div']
divisions


,lg,div
SFN,N,NW
LAN,N,NW
TBA,A,AE
MIL,N,NC
HOU,A,AW
CHA,A,AC
BOS,A,AE
NYA,A,AE
TOR,A,AE
OAK,A,AW


In [4]:
standings = compute_standings(df)
# standings['div'] = divisions['div']
# standings['lg'] = divisions['lg']
standings

,W,L
SFN,107,55
LAN,106,56
TBA,100,62
HOU,95,67
MIL,95,67
CHA,93,69
BOS,92,70
NYA,92,70
TOR,91,71
SEA,90,72


In [5]:
# max_wins
162 -  standings['L']


SFN    107
LAN    106
TBA    100
HOU     95
MIL     95
CHA     93
BOS     92
NYA     92
TOR     91
SEA     90
SLN     90
ATL     89
OAK     86
CIN     83
PHI     82
CLE     80
SDN     79
DET     77
ANA     77
NYN     77
KCA     74
COL     75
MIN     73
CHN     72
MIA     67
WAS     65
TEX     60
PIT     61
ARI     52
BAL     52
Name: L, dtype: int64

In [6]:
# standings.loc[(standings['div'] == 'NE')]['W'].max()

def division_contenders(standings_immutable):
  df = standings_immutable.copy()
  df['div'] = divisions['div']
  max_wins = 162 - df['L']
  division_contenders = set()
  for index, row in df.iterrows():
    most_wins_in_division = df.loc[(df['div'] == row["div"])]['W'].max()
    if max_wins[index] >= most_wins_in_division:
      division_contenders.add(index)
  return division_contenders
print(f'Still in contention: {division_contenders(standings)}')

Still in contention: {'MIL', 'CHA', 'SFN', 'ATL', 'HOU', 'TBA'}


In [7]:
from datetime import datetime, timedelta

def retro_to_datetime(retro_str):
    return datetime.strptime(retro_str, '%Y%m%d')

def datetime_to_retro(dt):
    return datetime.strftime(dt, '%Y%m%d')

def show_dumb_elimination_output(df, divisions):
    max_date = retro_to_datetime(df['completion_date'].max())
    min_date = retro_to_datetime(df['completion_date'].min())

    current_date = max_date
    contenders = set()
    old_contenders = None
    while current_date > min_date and len(contenders) < 30:
      date_str = datetime_to_retro(current_date)
      current_standings = compute_standings(df.loc[df['completion_date'] <= date_str])
      # MOVE THIS SHIT INTO SOMETHING EFFICIENT
      current_standings['div'] = divisions['div']
      current_standings['lg'] = divisions['lg']
      current_standings['max_wins'] = 162 - current_standings['L']
      contenders = division_contenders(current_standings)
      if old_contenders:
        new_contenders = contenders.difference(old_contenders)
        if new_contenders:
          print(f'Teams eliminated from their division titles on {datetime_to_retro(tomorrow)}: {new_contenders}')
      tomorrow = current_date
      old_contenders = contenders.copy()
      current_date = current_date - timedelta(days=1)

show_dumb_elimination_output(df, divisions)

Teams eliminated from their division titles on 20211003: {'LAN'}
Teams eliminated from their division titles on 20210930: {'SEA', 'PHI'}
Teams eliminated from their division titles on 20210927: {'OAK'}
Teams eliminated from their division titles on 20210926: {'SLN'}
Teams eliminated from their division titles on 20210925: {'NYA', 'BOS', 'NYN'}
Teams eliminated from their division titles on 20210924: {'TOR'}
Teams eliminated from their division titles on 20210923: {'CLE'}
Teams eliminated from their division titles on 20210919: {'MIA', 'DET'}
Teams eliminated from their division titles on 20210918: {'ANA', 'CIN'}
Teams eliminated from their division titles on 20210917: {'WAS'}
Teams eliminated from their division titles on 20210916: {'KCA'}
Teams eliminated from their division titles on 20210913: {'MIN', 'SDN'}
Teams eliminated from their division titles on 20210910: {'CHN'}
Teams eliminated from their division titles on 20210906: {'COL'}
Teams eliminated from their division titles on 2

In [8]:
def load_schedule(filename):
  schedule_columns=['date', 'doubleheader_index','weekday', 'visitor', 'visitor_league', 'visitor_game_num', 'home', 'home_league', 'home_game_num', 'day_night', 'completion', 'makeup_date']
  return pd.read_csv(filename, header=0, names=schedule_columns, usecols=range(len(schedule_columns)))

schedule_path = './data/2021SKED.TXT'

def find_unplayed_games(schedule):
  return schedule.loc[schedule["makeup_date"].str.startswith("not", na=False)]

find_unplayed_games(load_schedule(schedule_path))

,date,doubleheader_index,weekday,visitor,visitor_league,visitor_game_num,home,home_league,home_game_num,day_night,completion,makeup_date
2189,20210916,0,Thu,COL,NL,147,ATL,NL,146,d,Inclement weather,not rescheduled (game was not played)


In [9]:
def logged_games_after_date(df, date_str):
    return df.loc[df['completion_date'] > date_str]

def all_matchups_after_date(df, schedule, date_str):
    logged_games = logged_games_after_date(df, date_str)[['date', 'doubleheader_index', 'visitor','home']]
    unplayed_games = find_unplayed_games(schedule)[['date', 'doubleheader_index', 'visitor','home']]
    all_games = pd.concat([logged_games, unplayed_games], ignore_index=True)
    alpha_pairs = pd.DataFrame(np.where(all_games['visitor'] < all_games['home'],
                                     (all_games['visitor'], all_games['home']),
                                     (all_games['home'], all_games['visitor'])))
    return alpha_pairs.T.rename(columns={0: 'alpha1', 1: 'alpha2'})

all_matchups_after_date(df, load_schedule(schedule_path), '20211001').groupby(['alpha1', 'alpha2'], as_index=False).size()


,alpha1,alpha2,size
0,ANA,SEA,2
1,ARI,COL,2
2,ATL,COL,1
3,ATL,NYN,2
4,BAL,TOR,2
5,BOS,WAS,2
6,CHA,DET,2
7,CHN,SLN,2
8,CIN,PIT,2
9,CLE,TEX,2


In [10]:

def divisional_threats(standings_immutable, team):
    # We only care about teams with more max_wins than us.
    df = standings_immutable.copy()
    my_division = divisions.loc[team]['div']
    df = df.merge(divisions, left_index=True, right_index=True).loc[divisions['div'] == my_division]   
    max_wins = 162 - df['L']
    df = df.loc[(df.index != team) & (max_wins[df.index] >= max_wins[team])]
    return max_wins[team] - df['W']
#    return df.loc[(df.index != team) & (divisions.loc[df.index] == my_division) & (max_wins[df.index] >= max_wins[team])]
    # return df.merge(divisions, left_index=True, right_index=True(df.index != team) & (max_wins[df.index] >= max_wins[team])]
#standings.loc['NYN']['div']
divisional_threats(standings, 'NYN')
#standings.merge(divisions, left_index=True, right_index=True).loc[divisions['div'] == 'NE']

ATL   -11
PHI    -5
Name: W, dtype: int64

In [32]:
orioles_test = compute_standings(df.loc[(df['completion_date'] <= '20210823')])
threats1 = divisional_threats(orioles_test, 'BAL')
threats1['NYA']
# why_cant_i_do_this_in_one_line.merge(divisions, left_index=True, right_index=True)
# orioles_test = why_cant_i_do# _this_in_one_line.loc[divisions[why_cant_i_do_this_in_one_line.index] == 'AE']

4

In [38]:
def games_between_rivals_after_date(df, schedule, date_str, team):
    remaining = all_matchups_after_date(df, schedule, date_str).groupby(['alpha1', 'alpha2'], as_index=False).size()
    standings = compute_standings(df.loc[(df['completion_date'] <= date_str)])
    threats = divisional_threats(standings, team)
    return remaining.loc[(remaining['alpha1'].isin(threats.index)) & (remaining['alpha2'].isin(threats.index))]



In [36]:
# I have to sort games_among_threats in decreasing order of team quality
# Maybe just replace the team names with ints?


In [67]:
# sort the output of division_threats
def sort_rivals(matchups, threats):
    df = matchups.copy()
    merge1 = df.merge(threats, left_on=['alpha1'], right_index=True)
    merge2 = merge1.merge(threats, left_on=['alpha2'], right_index=True)
    merge2['betterT'] = np.where(merge2['W_x'] >= merge2['W_y'], merge2['alpha2'], merge2['alpha1'])
    merge2['worseT'] = np.where(merge2['W_x'] >= merge2['W_y'], merge2['alpha1'], merge2['alpha2'])
    merge2['lesserW'] = merge2[['W_x','W_y']].min(axis=1)
    merge2['greaterW'] = merge2[['W_x','W_y']].max(axis=1)
    return merge2.sort_values(['lesserW', 'greaterW'], ascending=True)[['betterT', 'worseT', 'size']]
    
matchups1 = games_between_rivals_after_date(df, load_schedule(schedule_path), '20210823', 'BAL')
threats1 = divisional_threats(compute_standings(df.loc[(df['completion_date'] <= '20210823')]), 'BAL')
sorted_rivals1 = sort_rivals(matchups1, threats1)
sorted_rivals1

,betterT,worseT,size
112,TBA,NYA,3
40,TBA,BOS,7
129,TBA,TOR,6
37,NYA,BOS,3
114,NYA,TOR,7


In [78]:
def portion_out_wins(sorted_rivals, threats):
    for _, row in sorted_rivals.iterrows():
        games = row['size']
        betterT = row['betterT']
        worseT = row['worseT']
        threat1, threat2 = threats[betterT], threats[worseT]
  #      print(f'We can afford to let {betterT} win {threat1} more games, and {worseT} {threat2}')
        if games > threat1 + threat2:
            # No matter what, these games will lead to too many wins for one team.
#            print(f'But there are {games} games left between {betterT} and {worseT}')
            return False
        # We try to give as many wins to the best team as possible first. Then distribute them
        # downward. Not sure about this.
        # In the Baltimore example, Tampa Bay cannot win another game. So wins_for_t1 is going to be
        # zero. We give all three of the TB-NY series to NY.
        wins_for_t2 = min(threat2, games)
        wins_for_t1 = games - wins_for_t2
#        print(f'Let\'s say that {betterT} wins {wins_for_t1} of their remaining {games} with {worseT}')
        threats[row['betterT']] -= wins_for_t1
        threats[row['worseT']] -= wins_for_t2
    return True

matchups1 = games_between_rivals_after_date(df, load_schedule(schedule_path), '20210822', 'PHI')
threats1 = divisional_threats(compute_standings(df.loc[(df['completion_date'] <= '20210822')]), 'PHI')
sorted_rivals1 = sort_rivals(matchups1, threats1)
threats1
# portion_out_wins(sorted_rivals1, threats1)                                                           

ATL    33
Name: W, dtype: int64

In [79]:
def is_division_contender_with_rivalries(game_log, schedule, date_str, team):
    matchups = games_between_rivals_after_date(game_log, schedule, date_str, team)
    threats = divisional_threats(compute_standings(game_log.loc[(game_log['completion_date'] <= date_str)]), team)
    sorted_rivals = sort_rivals(matchups, threats)
    return portion_out_wins(sorted_rivals, threats)

SCHEDULE = load_schedule(schedule_path)
is_division_contender_with_rivalries(df, SCHEDULE, '20210822', 'BAL')

False

In [95]:
def show_slightly_less_dumb_elimination_output(df, schedule, divisions):
    max_date = retro_to_datetime(df['completion_date'].max())
    min_date = retro_to_datetime(df['completion_date'].min())

    current_date = max_date
    contenders = set()
    old_contenders = None
    while current_date > min_date and len(contenders) < 30:
      date_str = datetime_to_retro(current_date)
      current_standings = compute_standings(df.loc[df['completion_date'] <= date_str])
      # MOVE THIS SHIT INTO SOMETHING EFFICIENT
      current_standings['div'] = divisions['div']
      current_standings['lg'] = divisions['lg']
      current_standings['max_wins'] = 162 - current_standings['L']
      contenders = division_contenders(current_standings)
      for supposed_contender in contenders.copy():
            # If you're in contention tomorrow, you're in contention today, so I am not going to
            # waste CPU time on you.
            if old_contenders and supposed_contender not in old_contenders:
                if not is_division_contender_with_rivalries(df, schedule, date_str, supposed_contender):
                    print(f'It looked like {supposed_contender} was in contention after {date_str} but the remaining intra-division games ruled them out.')
                    contenders.remove(supposed_contender)
      if old_contenders:
        new_contenders = contenders.difference(old_contenders)
        if new_contenders:
          print(f'Teams eliminated from their division titles on {datetime_to_retro(tomorrow)}: {new_contenders}')
      tomorrow = current_date
      old_contenders = contenders.copy()
      current_date = current_date - timedelta(days=1)

show_slightly_less_dumb_elimination_output(df, SCHEDULE, divisions)

Teams eliminated from their division titles on 20211003: {'LAN'}
Teams eliminated from their division titles on 20210930: {'SEA', 'PHI'}
Teams eliminated from their division titles on 20210927: {'OAK'}
Teams eliminated from their division titles on 20210926: {'SLN'}
Teams eliminated from their division titles on 20210925: {'NYA', 'BOS', 'NYN'}
Teams eliminated from their division titles on 20210924: {'TOR'}
Teams eliminated from their division titles on 20210923: {'CLE'}
It looked like MIA was in contention after 20210918 but the remaining intra-division games ruled them out.
Teams eliminated from their division titles on 20210919: {'DET'}
Teams eliminated from their division titles on 20210918: {'ANA', 'MIA', 'CIN'}
It looked like WAS was in contention after 20210916 but the remaining intra-division games ruled them out.
It looked like WAS was in contention after 20210915 but the remaining intra-division games ruled them out.
Teams eliminated from their division titles on 20210916: {'

In [ ]:
def wildcard_contenders(standings_immutable):
  df = standings_immutable.copy()
  df['div'] = divisions['lg']
  max_wins = 162 - df['L']
  division_contenders = set()
  for index, row in df.iterrows():
    most_wins_in_division = df.loc[(df['div'] == row["div"])]['W'].max()
    if max_wins[index] >= most_wins_in_division:
      division_contenders.add(index)
  return division_contenders
print(f'Still in contention: {division_contenders(standings)}')